In [15]:
from random import seed
from random import randrange
from csv import reader
import numpy as np
from sklearn.model_selection import train_test_split
import math
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [16]:
class DecisionTree:
    def __init__(self, maxDepth,partitions,minSize):
        self.maxDepth = maxDepth
        self.partitions = partitions
        self.minSize = minSize
         
    # Calculate accuracy percentage
    def calculateAccuracyMetrics(self,actual, predicted):
        correct = 0
        for i in range(len(actual)):
            if actual[i] == predicted[i]:
                correct += 1
        return correct / float(len(actual)) * 100.0
    
    def evaluatePerformance(self,y_test, predictedLabelList):
        data = {'Predicted': predictedLabelList ,
                'Actual':  y_test  
               }

        df = pd.DataFrame(data, columns=['Actual','Predicted'])

        confusion_matrix = pd.crosstab(df['Actual'], df['Predicted'])
        print("\nConfusion Matrix:")
        print(confusion_matrix) 

        tn=confusion_matrix[0][0]
        fn=confusion_matrix[0][1]
        fp=confusion_matrix[1][0]
        tp=confusion_matrix[1][1]

        Accuracy=(tp+tn)/(tp+tn+fp+fn)*100
        Precision=(tp)/(tp+fp)
        Recall=(tp)/(tp+fn)
        F1_Score= 2*((Recall*Precision)/(Recall + Precision))
        #print("TP:",tp," TN:",tn," FN:",fn," FP:",fp)
        return Accuracy,Precision,Recall,F1_Score
    
    # Evaluate an algorithm using a cross validation split
    def fit(self,train_set,test_set):
        predicted = self.getDecisionTree(train_set, test_set)
        actual = np.array(test_set)[:,len(test_set[0])-1]
        Accuracy,Precision,Recall,F1_Score = self.evaluatePerformance(actual, predicted)
        return Accuracy,Precision,Recall,F1_Score
    
    # Split a dataset based on an attribute and an attribute value
    def doRandomSplit(self,index, value, dataset):
        left = []
        right = []
        _ = [left.append(row) if row[index] < value else right.append(row) for row in dataset]
        return left, right
    
    # Calculate the Gini index for a split dataset
    def calGiniIndex(self,groups, labels):
        # total gives the summation of all the instances combining all groups
        total = float(sum([len(group) for group in groups]))
        # initialise gini to zero
        gini = 0.0
        for group in groups:
            # gives the elements belonging to each of the group
            group_size = float(len(group))
            # ignores if the size is 0
            if group_size == 0:
                continue
            # initializing score to zero
            score = 0.0
            for label in labels:
                # calculate the score for each of the data split
                p = [row[-1] for row in group].count(label) / group_size
                score += p * p
            # weight the group score by its relative size
            gini += (1.0 - score) * (group_size / total)
        return gini
    
    # Select the best split point for a dataset
    def findBestSplit(self,dataset):
        # list pf different labels in the last column of the data
        labels = list(set(row[-1] for row in dataset))
        b_index, b_value, b_score, b_groups = 999, 999, 999, None
        for index in range(len(dataset[0])-1):
            for row in dataset:
                groups = self.doRandomSplit(index, row[index], dataset)
                gini = self.calGiniIndex(groups, labels)
                if gini < b_score:
                    #When selecting the best split and using it as a new node for the tree we will
                    #store the index of the chosen attribute, the value of that attribute by 
                    #which to split and the two groups of data split by the chosen split point
                    b_index, b_value, b_score, b_groups = index, row[index], gini, groups
        return {'index':b_index, 'value':b_value, 'groups':b_groups, 'gini':b_score}
    
    # Create a terminal node value
    def leaf(self,group):
        outcomes = [row[-1] for row in group]
        return max(set(outcomes), key=outcomes.count)
    
    # Create child splits for a node or make leaf
    def split(self,node,depth):
        # the two groups of data split by the node are extracted for use
        left, right = node['groups']
        # check for a no split
        # if either the left group or the right group is empty then we assign it as the leaf node
        if not left or not right:
            node['left'] = node['right'] = self.leaf(left + right)
            return
        # check for max depth if the max depth is reached we agian create the leaf node
        if depth >= self.maxDepth:
            node['left'], node['right'] = self.leaf(left), self.leaf(right)
            return
        # We check if the the group of rows is too small . If it is too small then we again assign it as a leaf node
        if len(left) <= self.minSize:
            node['left'] = self.leaf(left)
        else:
        # if non of the conditions satisfy we recursively split the node in a similar fashion.
            node['left'] = self.findBestSplit(left)
            self.split(node['left'],depth+1)
        # We check if the number of rows in each group is too small include it as aleaf node
        if len(right) <= self.minSize:
            node['right'] = self.leaf(right)
        else:
        # if non of the above conditions satisfy we recursively split the data again
            node['right'] = self.findBestSplit(right)
            self.split(node['right'], depth+1)
            
    # Build a decision tree
    def buildDecisionTree(self,train):
        root = self.findBestSplit(train)
        self.split(root, 1)
        return root
    
    # Make a prediction with a decision tree
    def predict(self,node, row):
        #We must check if a child node is either a terminal value to be returned as the prediction, 
        #or if it is a dictionary node containing another level of the tree to be considered.
        if row[node['index']] < node['value']:
            # checks if the object is an instance of another instance
            if isinstance(node['left'], dict):
                return self.predict(node['left'], row)
            else:
                return node['left']
        else:
            # checks if the object is an instance of another instance
            if isinstance(node['right'], dict):
                return self.predict(node['right'], row)
            else:
                return node['right']
            
    # Classification and Regression Tree Algorithm
    def getDecisionTree(self,train, test):
        tree = self.buildDecisionTree(train)
        print("\n ***********Decision Tree***********\n")
        print('Best Split for Feature%d < %.3f Gini=%.3f' % ((tree['index']+1), tree['value'], tree['gini']))
        print("\n")
        self.printTree(tree,self.maxDepth)
        predictions = list()
        for row in test:
            prediction = self.predict(tree, row)
            predictions.append(prediction)
        return(predictions)
    
    # Print a decision tree
    def printTree(self,node, depth=0,level=0):
        if isinstance(node, dict):
            print('%s%s%d%s[Feature %d < %.3f]' % ((depth*' ','Level ',level+1,':', (node['index']+1), node['value'])))
            self.printTree(node['left'], depth+1,level+1)
            self.printTree(node['right'], depth+1,level+1)
        else:
            print('%s[%s]' % (((depth+7)*' ', node)))

In [20]:
def convertToFloat(sequence):
    for item in sequence:
        try:
            yield float(item)
        except ValueError as e:
            yield item

In [21]:
# Split a dataset into n partitions so that we can calculate the cost based on different splits
def doCrossValidation(dataset,partitions):
    splits = list()
    datasetCopy = list(dataset)
    chunkSize = len(datasetCopy) // partitions
    leftOver = len(datasetCopy) % partitions
    start = 0
    for i in range(partitions):
        if i < leftOver:
            end = start + chunkSize + 1
        else:
            end = start + chunkSize
        splits.append(datasetCopy[start:end])
        start = end
    return splits 

In [22]:
# Test CART on project 3 dataset
accuracy = []
precision = []
recall = []
fmeasure = []

# initialization of the partitions and the max depth of the trees
numPartitions = 10
seed(11)
# max_depth and min_size tells us when we stop growing the tree
maxDepth = 3

crossVal = False
minimumSize = 1

original_data = pd.read_csv('project3_dataset2.txt', delimiter="\t", header=None)
#print(original_data.iloc[0])
obj_df = original_data.select_dtypes(include=['object']).copy()
obj_df = obj_df.apply(LabelEncoder().fit_transform)
for col in obj_df:
    original_data[col] = obj_df[col]
#print(original_data.iloc[0])
dataset = original_data.values
#data = dataset.tolist()
#data = [list(convertToFloat(sublist)) for sublist in dataset]

decisionTreeData = dataset
# evaluate algorithm
dt = DecisionTree(maxDepth,numPartitions,minimumSize)
np.random.seed(11)
np.random.shuffle(decisionTreeData)
lent = len(dataset)
trrows = math.floor(0.8*lent)
train_set = decisionTreeData[0:trrows]
test_set = decisionTreeData[trrows:len(decisionTreeData)-1]
Accuracy,Precision,Recall,F1_Score = dt.fit(train_set,test_set)
recall.append(Recall)
precision.append(Precision)
fmeasure.append(F1_Score)
accuracy.append(Accuracy)

#print('Scores: %s' % scores)
#print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

mean_accuracy = np.mean(accuracy)
mean_precision = np.mean(precision)
mean_recall = np.mean(recall)
mean_fmeasure = np.mean(fmeasure)

print("\n \nMean accuracy is : ",mean_accuracy)
print("Mean precision is : ",mean_precision)
print("Mean recall is : ",mean_recall)
print("Mean fmeasure is : ",mean_fmeasure)


 ***********Decision Tree***********

Best Split for Feature9 < 51.000 Gini=0.402


   Level 1:[Feature 9 < 51.000]
    Level 2:[Feature 2 < 0.520]
     Level 3:[Feature 9 < 36.000]
             [0.0]
             [0.0]
     Level 3:[Feature 6 < 69.000]
             [0.0]
             [1.0]
    Level 2:[Feature 5 < 1.000]
     Level 3:[Feature 2 < 7.770]
             [0.0]
             [1.0]
     Level 3:[Feature 3 < 5.100]
             [1.0]
             [1.0]

Confusion Matrix:
Predicted  0.0  1.0
Actual             
0.0         55    6
1.0         16   15

 
Mean accuracy is :  76.08695652173914
Mean precision is :  0.7142857142857143
Mean recall is :  0.4838709677419355
Mean fmeasure is :  0.5769230769230769


# K Fold Validation

In [23]:
numPartitions = 10
seed(11)
maxDepth = 3
minimumSize = 1
kfoldData = dataset
accuracy = []
precision = []
recall = []
fmeasure = []
np.random.seed(11)
np.random.shuffle(kfoldData)
folds = doCrossValidation(kfoldData,numPartitions)

kdt = DecisionTree(maxDepth,numPartitions,minimumSize)

for index,fold in enumerate(folds):
    ktrain_set = list(folds)
    ktrain_set.pop(index)
    ktrain_set = sum(ktrain_set, [])
    ktest_set = fold
    Accuracy,Precision,Recall,F1_Score = kdt.fit(ktrain_set,ktest_set)
    recall.append(Recall)
    precision.append(Precision)
    fmeasure.append(F1_Score)
    accuracy.append(Accuracy)
    print("Accuracy: ",Accuracy)
    
mean_accuracy = np.mean(accuracy)
mean_precision = np.mean(precision)
mean_recall = np.mean(recall)
mean_fmeasure = np.mean(fmeasure)

print("\n \nMean accuracy is : ",mean_accuracy)
print("Mean precision is : ",mean_precision)
print("Mean recall is : ",mean_recall)
print("Mean fmeasure is : ",mean_fmeasure)


 ***********Decision Tree***********

Best Split for Feature9 < 49.000 Gini=0.406


   Level 1:[Feature 9 < 49.000]
    Level 2:[Feature 9 < 31.000]
     Level 3:[Feature 2 < 0.520]
             [0.0]
             [0.0]
     Level 3:[Feature 6 < 69.000]
             [0.0]
             [1.0]
    Level 2:[Feature 5 < 1.000]
     Level 3:[Feature 2 < 7.770]
             [0.0]
             [1.0]
     Level 3:[Feature 3 < 2.460]
             [0.0]
             [1.0]

Confusion Matrix:
Predicted  0.0  1.0
Actual             
0.0         29    4
1.0          5    9
Accuracy:  80.85106382978722

 ***********Decision Tree***********

Best Split for Feature9 < 51.000 Gini=0.397


   Level 1:[Feature 9 < 51.000]
    Level 2:[Feature 9 < 31.000]
     Level 3:[Feature 2 < 0.520]
             [0.0]
             [0.0]
     Level 3:[Feature 6 < 69.000]
             [0.0]
             [1.0]
    Level 2:[Feature 5 < 1.000]
     Level 3:[Feature 2 < 7.770]
             [0.0]
             [1.0]
     Leve